### Sammensette
#### Funksjon for å slå sammen datasett.
<font size=2>I pyspark kan man bruke funksjonene:
<code>unionByName</code>:<i>Return a new SparkDataFrame containing the union of rows, matched by column names. This is different from union function, and both UNION ALL and UNION DISTINCT in SQL as column positions are not taken into account. Input SparkDataFrames can have different data types in the schema.</i> ([spark.apache.org/unionByName](https://spark.apache.org/docs/latest/api/R/unionByName.html))
<code>union</code>: <i>Return a new SparkDataFrame containing the union of rows. This is equivalent to UNION ALL in SQL. Input SparkDataFrames can have different schemas (names and data types).</i> ([spark.apache.org/union](https://spark.apache.org/docs/latest/api/R/union.html))
</font>

<font size=2>I eksemplene tar vi utgangspunkt i datasettene:
- Innbyggerantalldatasett 2020 (df_land_innbyggerantall_2020) med variablene Land, Landkode, Innbyggerantall, År og Kilde definert i nevnte rekkefølge.
- Innbyggerantalldatasett 2018 (df_land_innbyggerantall_2018) med variablene Land, Landkode, Innbyggerantall, År og Kilde definert i nevnte rekkefølge.
- Innbyggerantalldatasett 2017 (df_land_innbyggerantall_2017) med variablene Landkode, Land, Innbyggerantall, År og Kilde definert i nevnte rekkefølge. (<b>Merk</b> at rekkefølge er ulik fra 2018 og 2020 datasett)

#### Eksempler
<ol>
<li>Sette sammen innbyggerantall_2018 og innbyggerantall_2017 med <code>unionByName</code> 
<li>Sette sammen innbyggerantall_2018 og innbyggerantall_2017 med <code>union</code>(eksempel illustrerer bruk av union på datasett hvor schema definisjon og rekkefølge på variable ikke er lik . Resultat blir <b>feil</b>)
<li>Sette sammen utsnitt (kun rader med landkode = 'NO') fra innbyggerantall 2020 og innbyggerantall 2018 med <code>union</code>. (eksempel illustrerer bruk av union på datasett hvor schema definisjon og rekkefølge på variable er lik.
<li>Sette sammen utsnitt fra innbyggerantall 2020 datasett med verson av innbyggerantall 2018 datasett som har fått tilført en ny variabel InnbyggerAntI1000. 
<li>Sette sammen datasettene med egen funksjon <code>unequal_union_sdf</code>. Datsettene som settes sammen har ulikt antall variable og følgelig forskjellig skjema. 
</ol>
</font>

#### Import av bibliotek (kode)

In [ ]:
from pyspark.sql import DataFrame
from pyspark.sql.types import *
import pyspark.sql.functions as f

#### Lese inn data i eksempeldata (kode)

In [ ]:
df_land_innbyggerantall_2020  = spark.read.path("gsim").load("/felles/veiledning/pyspark/eksempler/innbyggerantall/2020")
df_land_innbyggerantall_2018  = spark.read.path("gsim").load("/felles/veiledning/pyspark/eksempler/innbyggerantall/2018")
df_land_innbyggerantall_2017  = spark.read.path("gsim").load("/felles/veiledning/pyspark/eksempler/innbyggerantall/2017")

### Likheter og forskjeller i datasettene
<font size=2>Datasettene innbyggerantall 2017 og 2018 inneholder de samme variablene, men <code>Landkode</code> og <code>Land</code> er definert i omvendt rekkefølge. I tillegg har variabel <code>År</code> forskjellig datatype - <code>Integertype</code> i 2018 settet og <code>Stringtype</code> i 2017.
</font>

#### Utskrift av df_land_innbyggerantall_2017 (kode)

In [ ]:
df_land_innbyggerantall_2017.printSchema()
df_land_innbyggerantall_2017.show(5,False)

#### Utskrift av df_land_innbyggerantall_2018 (kode)

In [ ]:
df_land_innbyggerantall_2018.printSchema()
df_land_innbyggerantall_2018.show(5,False)

#### Eksempel 1- unionByName: Sette sammen innbyggerantall_2018 og innbyggerantall_2017 med unionByName (kode)

In [ ]:
df_land_innbyggerantall_2018_2017 = df_land_innbyggerantall_2018.unionByName(df_land_innbyggerantall_2017)
df_land_innbyggerantall_2018_2017.show(10,False)
antall_rader_2018       = df_land_innbyggerantall_2018.count()
antall_rader_2017       = df_land_innbyggerantall_2017.count()
antall_rader_2018_2017  = df_land_innbyggerantall_2018_2017.count()

print("antall rader i dataframe df_land_innbyggerantall 2018      : "+ repr(antall_rader_2018))
print("antall rader i dataframe df_land_innbyggerantall 2017      : "+ repr(antall_rader_2017))
print("antall rader i dataframe sammensatt                        : "+ repr(antall_rader_2018_2017))
print(" ")
#df_land_innbyggerantall_2018_2017.printSchema()

#### Eksempel 2 - union: Sette sammen innbyggerantall_2018 og innbyggerantall_2017 med union (kode)
Eksempel illustrerer bruk av union på datasett hvor der rekkefølge på variable ikke er lik. Resultat blir feil.

In [ ]:
df_land_innbyggerantall_2018_2017 = df_land_innbyggerantall_2018.union(df_land_innbyggerantall_2017)
df_land_innbyggerantall_2018_2017.show(10, False)
antall_rader_2018       = df_land_innbyggerantall_2018.count()
antall_rader_2017       = df_land_innbyggerantall_2017.count()
antall_rader_2018_2017  = df_land_innbyggerantall_2018_2017.count()


print("antall rader i dataframe df_land_innbyggerantall 2018      : "+ repr(antall_rader_2018))
print("antall rader i dataframe df_land_innbyggerantall 2017      : "+ repr(antall_rader_2017))
print("antall rader i dataframe sammensatt                        : "+ repr(antall_rader_2018_2017))

df_land_innbyggerantall_2018_2017.printSchema()

Filtrerer ut Norge fra innbyggerantall 2020 dataframe - df_land_innbyggerantall_2020_utsnitt - er input dataframe i påfølgende paragrafer

In [ ]:
df_land_innbyggerantall_2020_utsnitt = df_land_innbyggerantall.filter((df_land_innbyggerantall.Landkode == 'NO'))
df_land_innbyggerantall_2020_utsnitt.show(5, False)

#### Eksempel 3 - union: Sette sammen utsnitt (kun rader med landkode = 'NO') fra innbyggerantall 2020 og innbyggerantall 2018 med union. (kode)
Eksempel illustrerer bruk av union på datasett hvor schema definisjon og rekkefølge på variable er lik.

In [ ]:
df_land_innbyggerantall_utsnitt2020_2018 = df_land_innbyggerantall_2020_utsnitt.union(df_land_innbyggerantall_2018)
df_land_innbyggerantall_utsnitt2020_2018.show(10, False)
antall_rader_utsnitt2020 = df_land_innbyggerantall_2020_utsnitt.count()
antall_rader_2018         = df_land_innbyggerantall_2018.count()
antall_rader_utsnitt_2020_2018  = df_land_innbyggerantall_utsnitt2020_2018.count()
print("antall rader i dataframe utsnitt df_land_innbyggerantall 2020      : "+ repr(antall_rader_utsnitt2020))
print("antall rader i dataframe df_land_innbyggerantall 2018              : "+ repr(antall_rader_2018))
print("antall rader i dataframe sammensatt                                : "+ repr(antall_rader_utsnitt_2020_2018))

Legge til ny kolonne på innbyggerantall 2018 dataframe. Dataframe danner input til neste pargraf hvor vi setter sammen dataframes med ulikt antall variable sammen ved hjelp av egen funksjon

In [ ]:
df_land_innbyggerantall_2018_ny = df_land_innbyggerantall_2018.withColumn('InnbyggerAntI1000', df_land_innbyggerantall_2018.Innbyggerantall/1000)
df_land_innbyggerantall_2018_ny.show(5,False)

#### Eksempel 4 - Funksjon unequal_union_sdf (kode)
Sette sammen utsnitt fra innbyggerantall 2020 datasett med verson av innbyggerantall 2018 datasett som har fått tilført en ny variabel InnbyggerAntI1000. Setter sammen datasettene med egen funksjon  unequal_union_sdf. Datsettene som settes sammen har ulikt antall variable og følgelig forskjellig schema.

In [ ]:
#Funksjonen under er funnet/hentet fra stackowerflow.com
def unequal_union_sdf(sdf1, sdf2):
    s_df1_schema = set((x.name, x.dataType) for x in sdf1.schema)
    s_df2_schema = set((x.name, x.dataType) for x in sdf2.schema)
    for i,j in s_df2_schema.difference(s_df1_schema):
        sdf1 = sdf1.withColumn(i,f.lit(None).cast(j))
    for i,j in s_df1_schema.difference(s_df2_schema):
        sdf2 = sdf2.withColumn(i,f.lit(None).cast(j))
    
    common_schema_colnames = sdf1.columns
    sdk = \
        sdf1.select(common_schema_colnames).union(sdf2.select(common_schema_colnames))
    return sdk 
#
df_sammensatt = unequal_union_sdf(df_land_innbyggerantall_2020_utsnitt, df_land_innbyggerantall_2018_ny)
df_sammensatt.show(10, False)